In [1]:
import yfinance as yf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

## Lab 1: Dual Moving Average Crossover

**Dual Moving Average Crossover (Giao cắt trung bình động kép)** là một chiến lược giao dịch dựa trên hai đường trung bình động có chu kỳ khác nhau để xác định xu hướng thị trường.


## Fetch some stock data
Let's go with AAPL.

In [2]:
AAPL = yf.download("AAPL",period="2y",progress=False)
AAPL.columns = AAPL.columns.droplevel(1)
AAPL

YF.download() has changed argument auto_adjust default to True


Price,Close,High,Low,Open,Volume
Date,,,,,
2023-04-05,162.125015,163.402144,160.184592,163.095242,51511700
2023-04-06,163.016022,163.313030,160.382576,160.808275,45390100
2023-04-10,160.412292,160.412292,158.481764,159.808382,47716900
2023-04-11,159.194580,160.739003,158.907467,160.729108,47644200
2023-04-12,158.501572,160.441995,158.184759,159.610385,50133100
...,...,...,...,...,...
2025-03-31,222.130005,225.619995,216.229996,217.009995,65299300
2025-04-01,223.190002,223.679993,218.899994,219.809998,36412700
2025-04-02,223.889999,225.190002,221.020004,221.320007,35905900


## Visualize the stock price

In [3]:
fig = px.line(AAPL, y="Close", title='AAPL Stock Price', labels = {'Close':'AAPL Close Price(in USD)'})

In [4]:
fig.show()

## Moving Average 1 (Short window)

Here I am choosing Exponential moving average instead of Simple Moving Average, feel free to change it to SMA instead of EMA, you can do so in the following way.
```python
ema1['Close'] = AAPL['Close'].ewm(span = window1).mean()

```

In [5]:
window1 = 30
sma1 = pd.DataFrame()
sma1['Close'] = AAPL['Close'].rolling(window = window1).mean()
sma1

,Close
Date,
2023-04-05,NaN
2023-04-06,NaN
2023-04-10,NaN
2023-04-11,NaN
2023-04-12,NaN
...,...
2025-03-31,229.037334
2025-04-01,228.328000
2025-04-02,227.628667


## Moving Average 2 (Long Window)

In [6]:
window2 = 100
sma2 = pd.DataFrame()
sma2['Close'] = AAPL['Close'].rolling(window = window2).mean()
sma2

,Close
Date,
2023-04-05,NaN
2023-04-06,NaN
2023-04-10,NaN
2023-04-11,NaN
2023-04-12,NaN
...,...
2025-03-31,234.514269
2025-04-01,234.530945
2025-04-02,234.540252


In [7]:
fig.add_scatter(x=sma1.index,y=sma1['Close'], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=sma2.index,y=sma2['Close'], mode='lines',name='SMA'+str(window2))
fig.show()

## Combine everything

In [8]:
data = pd.DataFrame()
data['AAPL'] = AAPL['Close']
data['SMA'+str(window1)] = sma1['Close']
data['SMA'+str(window2)] = sma2['Close']
data

,AAPL,SMA30,SMA100
Date,,,
2023-04-05,162.125015,NaN,NaN
2023-04-06,163.016022,NaN,NaN
2023-04-10,160.412292,NaN,NaN
2023-04-11,159.194580,NaN,NaN
2023-04-12,158.501572,NaN,NaN
...,...,...,...
2025-03-31,222.130005,229.037334,234.514269
2025-04-01,223.190002,228.328000,234.530945
2025-04-02,223.889999,227.628667,234.540252


## Strategy to generate buy/sell signal

In [9]:
def dualMACrossover(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1 # Flag denoting when the 2 moving averages crossed each other
    for i in range(len(data)):
        if data['SMA'+str(window1)][i] > data['SMA'+str(window2)][i]:
            if flag != 1:
                sigPriceBuy.append(data['AAPL'][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data['SMA'+str(window1)][i] < data['SMA'+str(window2)][i]:
            if flag!=0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['AAPL'][i])
                flag=0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
    return (sigPriceBuy,sigPriceSell)

In [10]:
buy_sell = dualMACrossover(data)
data['BuySignalPrice'] = buy_sell[0]
data['SellSignalPrice'] = buy_sell[1]

C:\Users\HoaAn\AppData\Local\Temp\ipykernel_6604\2965301454.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HoaAn\AppData\Local\Temp\ipykernel_6604\2965301454.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HoaAn\AppData\Local\Temp\ipykernel_6604\2965301454.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\HoaAn\AppData\Local\Temp\ipykernel_6604\2965301454.py:17: FutureWarning:

Series.__getitem

## Visualize the data and the strategy

In [11]:
import plotly.graph_objects as go

fig = px.line(data, y="AAPL", title='Strategy Visualization', labels = {'index':'Date'})
fig.add_scatter(x=data.index,y=data['SMA'+str(window1)], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=data.index,y=data['SMA'+str(window2)], mode='lines',name='SMA'+str(window2))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.BuySignalPrice, marker_symbol='triangle-up',
                           marker_line_color="#000000", marker_color="#000000",
                           marker_line_width=2, marker_size=15, name='Buy'))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.SellSignalPrice, marker_symbol='triangle-down',
                           marker_line_color="#E74C3C", marker_color="#E74C3C",
                           marker_line_width=2, marker_size=15, name='Sell'))
fig.show()

## Backtest the strategy

In [12]:
# !pip install backtesting

In [13]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import backtesting
backtesting.set_bokeh_output(notebook=False)

c:\Users\HoaAn\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

c:\Users\HoaAn\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:55: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.



Loading BokehJS ...

In [14]:
class DualMACrossover(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, window1)
        self.ma2 = self.I(SMA, price, window2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(AAPL, DualMACrossover,
              exclusive_orders=True)
stats = bt.run()

In [15]:
stats

Start                     2023-04-05 00:00:00
End                       2025-04-04 00:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                     74.9004
Equity Final [$]                   9374.65488
Equity Peak [$]                   10402.32044
Return [%]                           -6.25345
Buy & Hold Return [%]                 5.31142
Return (Ann.) [%]                    -3.18965
Volatility (Ann.) [%]                21.47724
CAGR [%]                             -2.20451
Sharpe Ratio                         -0.14851
Sortino Ratio                        -0.21634
Calmar Ratio                         -0.10245
Alpha [%]                            -6.98028
Beta                                  0.13684
Max. Drawdown [%]                   -31.13504
Avg. Drawdown [%]                   -10.23045
Max. Drawdown Duration      526 days 00:00:00
Avg. Drawdown Duration      142 days 00:00:00
# Trades                                    6
Win Rate [%]                      

# Exercise
* What is the return and annual return of the above algorithm?
* What is the annual volatility of the stock?
* Backtest the same strategy for TESLA stock and calculate the return, annual return, and annual volatility
* Adjust the lengths of short window and long window (long window > short window) and see if you can increase the annual return